In [ ]:
'''
Basically, this is the order i have followed.
Run your preprocessor.py to create processed data document vectors
'''

In [1]:
from scipy.sparse import csr_matrix
from scipy.sparse import linalg
import numpy as np
import math

dfFile = 'processedData/word_df.txt'
docFile = 'processedData/documentVectors_train.txt'
docFile_test = 'processedData/documentVectors_test.txt'
#docFile = 'processedData/documentVectors.txt'

# Count the number of lines in the DocumentFrequency(df) file
def getNumUniqueWords():
	f = open(dfFile)
	count = 0
	for line in iter(f):
		count += 1
	f.close()
	return count

# Count the number of lines in the DocumentVector(docVectors) file
def getNumDocuments():
	f = open(docFile)
	count = 0
	for line in iter(f):
		count += 1
	f.close()
        print "count ", count
	return count

def updateProbDisributionUsingSimilarity(documents, centroids):
	probDistribution = []
	runningSum = 0.0

	for n in range(len(documents)):
		maxSim = 0.0000001
		# Find the cluster with whom the documents n has maximum similarity
		for k in range(len(centroids)):
			sim = computeSimilarity(documents[n], centroids[k])
			if (sim.nnz == 0):
				sim = 0.0000001
			else:
				sim = sim.data[0]

			if maxSim < sim:
				maxSim = sim
		# Take invserse of the maximum Similarity and update the probability distribution
		# This makes sure documents with lower similarity have higher probability of being sampled
		probDistribution.append(1.0/maxSim)
		runningSum += 1.0/maxSim

	probDistribution[:] = [x / runningSum for x in probDistribution]
	return probDistribution

# documents: The list of document vectors in the corpus
# centroids: The list of centroids
def updateProbDisribution(documents, centroids):
	probDistribution = []
	runningSum = 0.0
	for n in range(len(documents)):
		minDist = 9999999.0
		#Find the cluster at the closest distance from the document n
		for k in range(len(centroids)):
			diff = documents[n]-centroids[k]
			dist = diff.multiply(diff).sum()
			if dist < minDist:
				minDist = dist
		# Update the probability distribution so that documents with higher distance
		# have higher probability of being sampled.
		probDistribution.append(minDist)
		runningSum += minDist
	probDistribution[:] = [x / runningSum for x in probDistribution]
	return probDistribution

# This is the entry point for initializaing the centroids for KMeans++
# documents: The list of document vectors in the corpus
# numCentroid: Number of centroids that need to be initialized
# useSimMetric: decides whether to use the distance metric or similarity metric
def initCentroidKMeanspp(documents, numCentroid, useSimMetric):
	centroids = []
	# Initially define the probability distribution giving equal probability of being sampled to each document.
	probDistribution = [1.0/len(documents)] * len(documents)
	values = range(len(documents))
	for k in range(numCentroid):
		# Sample a document from the distribution
		docID = np.random.choice(values, 1, p=probDistribution)[0]
		# Add this document vector to the centroids list. In this case the centroids will be non-sparse.
		centroids.append(documents[docID])
		print("Initialized centroid: " + str(k+1))

		if useSimMetric is True:
			probDistribution = updateProbDisributionUsingSimilarity(documents, centroids)
		else:
			probDistribution = updateProbDisribution(documents, centroids)
	return centroids

# This is the entry point for randomly initializaing the centroids for KMeans
def randomInitCentroid(numCentroid, vectorSize):
	centroids = []
	for i in range(numCentroid):
		# Randomly Initialize a centroid. This will be a non-sparse vector
		centroidVector = np.random.random(vectorSize)
		# Normalize the centroid Vector and add it to the list of centroids
		centroids.append(centroidVector * 1/np.linalg.norm(centroidVector))
	return centroids

# Calculate the inverse Document Frequence of each term in the corpus.
def getInverseDocumentFrequencyMap(numDocuments):
	# A map to store the IDF values for each term in the corpus
	dfMap = {}
	f = open(dfFile)
	for line in iter(f):
		df = line.split(":")
		dfMap[df[0]] = numDocuments / int(df[1])
	return dfMap

# Initializing the document vectors by reading the data from the input document vector file
def getDocumentVectors(vectorSize):
	f = open(docFile)
	documents = []
	for line in iter(f):
		row = []
		col = []
		data = []
		wordFrequency = line.split()
		for word in wordFrequency:
			row.append(0)
			col.append(int(word.split(':')[0]))
			# Use only the term frequency to weigh each term in the document
			data.append(int(word.split(':')[1]))
		# Create the sparse document vector using the CSR_matrix data structure
		spVector = csr_matrix((data, (row, col)), shape=(1, vectorSize))
		# Normalize the document vector and add the normalized vector to the list of document vectors
		documents.append(spVector.multiply(1/np.sqrt(spVector.multiply(spVector).sum())))
	f.close()
	return documents

# Use the TF-IDF weights to initialize the document vectors
def getDocumentVectorsWithTFIDF(vectorSize,test=False):
	numDocuments = getNumDocuments()
	idf = getInverseDocumentFrequencyMap(numDocuments)
	if(test):
        f = open(docFile_test)
    else:
        f = open(docFile)
	documents = []
	for line in iter(f):
		row = []
		col = []
		data = []
		wordFrequency = line.split()
		for word in wordFrequency:
			row.append(0)
			col.append(int(word.split(':')[0]))
			# Use only the TF-IDF value to weigh each term in the document
                        if( idf[word.split(':')[0]] <=0):
                            print "Negative or zero", word, word.split(':'), idf[word.split(':')[0]]
			data.append(int(word.split(':')[1]) * math.log(idf[word.split(':')[0]]))
		# Create the sparse document vector using the CSR_matrix data structure
		spVector = csr_matrix((data, (row, col)), shape=(1, vectorSize))
		# Normalize the document vector and add the normalized vector to the list of document vectors
		documents.append(spVector.multiply(1/np.sqrt(spVector.multiply(spVector).sum())))
	f.close()
	return documents

# Compute Similarity between the document and the given centroid
def computeSimilarity(document, centroid):
	# Use the dot product as the similarity metric.
	# This is same as the cosine similarity because the document and centroid vectors have already been normalized before.
	sim = document.dot(centroid.T)[0]
	return sim

# Output the document-cluster assignment into the output file
def printDocumentClusters(clusterAssignment, outputFileName):
	docID = 1
	output = open(outputFileName, "w")
	for cluster in clusterAssignment:
		line = str(docID) + " " + str(cluster)
		output.write(line)
		output.write("\n")
		docID += 1
	output.close()

# Update the centroid by calculating the arithmetic mean of all the documents assigned to the cluster
def updateCentroid(documents, clusterAssignment, numClusters, vectorSize):
	centroidSum = np.zeros(shape=(numClusters,vectorSize))
	centroidCount = [0] * numClusters

	for n in range(len(clusterAssignment)):
		cluster = clusterAssignment[n]
		centroidSum[cluster] += documents[n]
		centroidCount[cluster] += 1

	centroids = []
	for k in range(numClusters):
		if centroidCount[k] > 0:
			# Find the arithmetic mean
			centroidVector = centroidSum[k] / centroidCount[k]
		else:
			# Empty Cluster! Randomly initialize the centroid in this case
			centroidVector = np.random.random((1, vectorSize))
		# Normalize the centorid before updating the centroids list
		centroids.append(centroidVector * 1/np.linalg.norm(centroidVector))

	return centroids

# Find the similarity of the documents with the cluster it is assigned to
def computeClusteringScore(clusterAssignment, centroids, documents):
	#Calculate Average Similarity of each Cluster
	centroidCount = [0] * len(centroids)
	centroidSimilarityCount = [0] * len(centroids)

	for n in range(len(clusterAssignment)):
		cluster = clusterAssignment[n]
		centroidCount[cluster] += 1
		centroidSimilarityCount[cluster] += computeSimilarity(documents[n], centroids[cluster])

	total = 0.0
	for k in range(len(centroids)):
		if centroidCount[k] == 0:
			avgSimilarity = 0
		else:
			avgSimilarity = centroidSimilarityCount[k] / centroidCount[k]
		total += avgSimilarity
	total = total / len(centroids)
	print "Average Similarity of the clusters: " + str(total)
	return total

# This is the main method that serves as the entry point to all the functionality
# numClusters: The number of clusters to be learnt
# useTFIDFWeight: This decides whether we need to use the TF-IDF weight or not while creating the document vector
# useKMeanspp: This decides whether we need to use KMeans++ or not to initialize the clusters
# useSimMetric: If we are using KMeans++, this parameter decides whether to use distance or similarity metric

def findClusterAssignment(numClusters, useTFIDFWeight=False, useKMeanspp=False, useSimMetric=False):
	vectorSize = getNumUniqueWords()
	documents = []
	centroids = []

	if useTFIDFWeight is True:
		print("Using TF-IDF weights for Document Vectors")
		documents = getDocumentVectorsWithTFIDF(vectorSize,False)
	else:
		print("Using TF weights for Document Vectors")
		documents = getDocumentVectors(vectorSize)

	if useKMeanspp == True:
		print("Using KMeans++ to initialize centroids")
		centroids = initCentroidKMeanspp(documents, numClusters, useSimMetric)
	else:
		print("Randomly initializing centroids")
		centroids = randomInitCentroid(numClusters, vectorSize)

	numDocs = len(documents)
	previousClusterAssignment = [-1] * numDocs
	clusterAssignment = [0] * numDocs

	numIterations = -1
	print("Running Loyd's Algorithm for clustering")
	while ( np.array_equal(previousClusterAssignment,clusterAssignment) == False):
		print("Iteration: " + str(numIterations))
		previousClusterAssignment = clusterAssignment[:]
		for n in range(numDocs):
			maxSim = 0.0
			cluster = clusterAssignment[n]
			for k in range(numClusters):
				sim = computeSimilarity(documents[n], centroids[k])
				if sim > maxSim:
					maxSim = sim
					cluster = k
			clusterAssignment[n] = cluster
		numIterations += 1
		centroids = updateCentroid(documents, clusterAssignment, numClusters, vectorSize)
	similarity = computeClusteringScore(clusterAssignment, centroids, documents)
	return clusterAssignment, centroids, similarity

# New functions for cluster details and count and stats
def getclustercount(centroids):
    keys=range(len(centroids))
    counts = dict.fromkeys(keys,0)
    # print counts
    f = open('output/assign.txt')
    for line in iter(f):
        cl =(line.strip().split(' ')[1].strip())
        cl = int(cl)
        if cl in counts:
            counts[cl] +=1
        else:
            counts[cl] = 1
    return counts

#take the list of centroids and cretae tuples of their mean, std, number of points assigned to them
def getclusterstats(centroids):
    cms={}
    counts = getclustercount(centroids)
    for i in range(len(centroids)):
    #     print "Id",i, "mean", np.mean(centroids[i]),"std",np.std(centroids[i])
        cms[i]=(np.mean(centroids[i]),np.std(centroids[i]),counts[i])
    return cms

#takes in a vector t, centroid vector, mean, std,outliercount, threshold parameter hp2
#and returns updated outlier count,and indication whether it s outlier
def is_out(t,centroid,cm, std,oc,hp2):
    outlier = False
    l= zip(np.nonzero(t)[0],np.nonzero(t)[1])
    for r,c in l:
#         print t[r,c]
        hih = (cm+ (hp2 * std) )
        low = (cm-(hp2 * std))
             
        if (t[r,c] > hih):
            oc+=1
            outlier = True
#             print "hi",hih, t[r,c]
            break
        if (t[r,c] < low):
            oc+=1
            outlier = True
#             print "lo",low, t[r,c]
            break
#     print "Escaped"
    return outlier, oc

In [2]:
ca,centroids,simi = findClusterAssignment(10,True,True, True)
printDocumentClusters(ca, 'output/assign.txt')

Using TF-IDF weights for Document Vectors
count  15000
found  1873 count 12928 15000


/home/epi/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:159: RuntimeWarning: divide by zero encountered in double_scalars


Using KMeans++ to initialize centroids
Initialized centroid: 1
Initialized centroid: 2
Initialized centroid: 3
Initialized centroid: 4
Initialized centroid: 5
Initialized centroid: 6
Initialized centroid: 7
Initialized centroid: 8
Initialized centroid: 9
Initialized centroid: 10
Running Loyd's Algorithm for clustering
Iteration: -1
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Average Similarity of the clusters: 0.491680165938


In [185]:
#hp2 is threshold parameter
#hp is outlier density parameter
#if you wnat to see hp vs numof clusters, just loop over hpl
hpl = np.arange(0.25,2.5,0.25)
# hpl = [0.8]
# hp = 2.0
hp2 = 3
hp2l = np.arange(3,6,1)

#tvs are vectors for streaming.
vectorsize= getNumUniqueWords()
tvs = getDocumentVectorsWithTFIDF(vectorsize,True)

for hp in hpl:
    #cms has mean std count for each centroid
    cms =  getclusterstats(centroids)
    #outlier count
    out_count = 0
    
    #centroid vs outliercount near to them
    out_dict = dict.fromkeys(range(len(centroids)),0)
    
    # number of evolving clusters
    evol = 0
    
    #tvs are vectors for streaming.
    for t in tvs:

        mins = 0.0
        for i in range(len(centroids)):
            d = computeSimilarity(t,centroids[i])
            if(d>=mins):
                mins= d
                cl=i
    #     print "closest cluster :", cl
        clu = centroids[cl]
        cm = cms[i][0]
        cstd = cms[i][1]
        outlier, out_count = is_out(t, clu, cm, cstd, out_count,hp2)
        
        if(not outlier):
    #         assign , basically increment count for that centroid
            ccc = cms[i][2]
            cms[i] = (cms[i][0],cms[i][1],ccc+1)
    #         print "i reached here",cms[i]
        else:
    #         print "outlier with respect to ",cl
            out_dict[cl]+=1
            if out_count >= (hp * cms[i][2]):
    #             print "---------\ntrigger re-cluster"
                evol+=1
    #             print "Number of evolved clusters", evol
    #             print "out_count", out_count, "cluster with max outlier proximity", max(out_dict, key=out_dict.get)
    #             print "----------re-cluster done-------"
                out_count= 0
                out_dict = dict.fromkeys(range(len(centroids)),0)
    print "Denisty",hp,"Number of clusters evolved", evol


3 	8
4 	8
5 	8
